# 处理product信息

In [1]:
import pandas as pd
import numpy as np
import datetime
import gc
import random

In [2]:
day_3_product = pd.read_csv('./day_3_product.csv')
day_30_product = pd.read_csv('./day_30_product.csv')

In [3]:
groups = day_3_product.groupby(['role_id'])
# 前3天充值总次数  p1
temp = groups.size().reset_index().rename(columns={0:'p1'})
matrix_1 = pd.merge(day_3_product, temp, on='role_id', how='left', validate='many_to_one')

# 前3天充值总金额  p2
temp = groups['money'].agg([('p2', 'sum')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')

# 前3天充值give_money次数 p3
temp = pd.DataFrame(groups['give_money'].apply(np.count_nonzero)).reset_index().rename(columns={'give_money':'p3'})
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')
# # 前3天充值give_money总金额 p4
temp = groups['give_money'].agg([('p4', 'sum')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')

# 前3天充值rmb_money次数 p5
temp = pd.DataFrame(groups['rmb_money'].apply(np.count_nonzero)).reset_index().rename(columns={'rmb_money':'p5'})
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')

# 前3天充值rmb_money总金额 p6
temp = groups['rmb_money'].agg([('p6', 'sum')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')


In [4]:
pd.set_option('display.max_columns', None)
matrix_1.head()

,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,pay_count,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt,is_pay,create_role_time,interval,p1,p2,p3,p4,p5,p6
0,12043074,106434579,0,128.0,0.0,128.0,0.0,0.0,NaN,0,1,1,53,BFD0278A-815B-4900-B3EB-AF106FAF66F0,2020-03-13 13:45:01,2020-03-13 13:45:01,1,2020-03-13,1,2020-03-10 22:00:37,63.740000,1,128.0,0,0.0,1,128.0
1,12063758,106557590,99,68.0,0.0,68.0,0.0,0.0,NaN,0,2,2,1,18C3BAF2-16E0-48AC-9999-8250815D8362,2020-03-13 00:01:21,2020-03-13 00:01:21,2,2020-03-13,1,2020-03-11 21:52:44,26.143611,5,246.0,0,0.0,5,246.0
2,12039463,106412872,2,30.0,0.0,30.0,0.0,0.0,NaN,0,3,3,0,76FF7593-8A26-454F-9A8E-7591FF81E5DD,2020-03-13 00:01:46,2020-03-13 00:01:46,3,2020-03-13,1,2020-03-10 19:19:25,52.705833,4,158.0,0,0.0,4,158.0
3,12084747,105791073,2,68.0,0.0,68.0,0.0,0.0,NaN,0,10,5,0,456AB5A6-DBC3-4B86-B624-487ED2E94E35,2020-03-13 00:02:02,2020-03-13 00:02:02,10,2020-03-13,1,2020-03-12 19:58:21,4.061389,9,608.0,0,0.0,9,608.0
4,12040203,106412684,2,68.0,0.0,68.0,0.0,0.0,NaN,0,6,6,1,971D99B6-1AFF-49D7-979E-D6B3AB8AF0DC,2020-03-13 00:05:49,2020-03-13 00:05:49,6,2020-03-13,1,2020-03-10 19:52:05,52.228889,6,276.0,0,0.0,6,276.0


In [5]:
# 选择一些字段进行保存
product_info_3d = matrix_1[['user_id','role_id','is_pay','p1','p2','p3','p4','p5','p6']]
product_info_3d = product_info_3d.drop_duplicates()
product_info_3d.to_csv('./FE_data/product_info_3d.csv', index=None)

# 处理login信息

In [2]:
login_info_3d = pd.read_csv('./login_info_3d.csv')

In [3]:
# 压缩数据
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
login_info_3d = reduce_mem_usage(login_info_3d)

Mem. usage decreased to 2175.03 Mb (17.3% reduction)


In [7]:
login_info_3d.head()

,a.user_id,a.role_id,a.login_time,a.role_level,a.device_no,a.ip,a.net_w,a.p_model,a.sys_ver,a.sdk_no,a.dt,b.role_id,b.create_role_time
0,96918420,10027198,2019-11-03 16:53:49,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,10027198,2019-11-03 16:50:00
1,96918420,10027198,2019-11-03 16:50:52,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,10027198,2019-11-03 16:50:00
2,96918420,10027198,2019-11-03 16:53:11,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,10027198,2019-11-03 16:50:00
3,96918420,10027198,2019-11-03 16:54:42,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,10027198,2019-11-03 16:50:00
4,96918420,10027198,2019-11-03 16:55:36,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,10027198,2019-11-03 16:50:00


In [12]:
login_info_3d[login_info_3d['a.role_id'] == 11405711].shape

(8, 13)

In [6]:
login_info_3d.drop('b.role_id', axis=1, inplace=True)
new_columns = [columns[2:] for columns in login_info_3d.columns]
login_info_3d.columns = new_columns
login_info_3d.head()

,user_id,role_id,login_time,role_level,device_no,ip,net_w,p_model,sys_ver,sdk_no,dt,create_role_time
0,96918420,10027198,2019-11-03 16:53:49,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
1,96918420,10027198,2019-11-03 16:50:52,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
2,96918420,10027198,2019-11-03 16:53:11,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
3,96918420,10027198,2019-11-03 16:54:42,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
4,96918420,10027198,2019-11-03 16:55:36,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00


In [11]:
# 角色3天内登陆的设备数，删除登陆设备大于等于8的role_id
groups = login_info_3d.groupby('role_id')
temp = groups['device_no'].agg([('device_no_num', 'nunique')]).reset_index().sort_values('device_no_num', ascending=False)
temp = temp[temp['device_no_num']>=8]
# 登陆设备大于等于8的role_id付费总金额
# print('登陆设备大于等于8的role_id付费总金额:' ,product_order[product_order['role_id'].isin(temp['role_id'])]['rmb_money'].sum())
login_info_3d.drop(login_info_3d[login_info_3d['role_id'].isin(temp['role_id'])].index, inplace=True)

In [7]:
# 按uesr_id,role_id进行聚合，查看user_id,role_id交互行为
groups = login_info_3d.groupby(['user_id','role_id'])
# user_id, role_id本周登录次数 r1
temp = groups.size().reset_index().rename(columns={0:'r1'})
matrix = pd.merge(login_info_3d, temp, on=['user_id','role_id'], how='left')
# user_id, role_id本周登录的设备数r2
temp = groups['device_no'].agg([('r2', 'nunique')]).reset_index()
matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')
# user,role_id本周使用的ip数r3
temp = groups['ip'].agg([('r3', 'nunique')]).reset_index()
matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')
# user,role_id本周登录的平均时间间隔r4 按小时
login_info_3d['login_time'] = pd.to_datetime(login_info_3d['login_time'])
temp = groups['login_time'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['r4'] = round((temp['L_time']-temp['F_time']).dt.total_seconds()/3600, 2)
matrix = pd.merge(matrix, temp[['user_id','role_id', 'r4']], on=['user_id','role_id'], how='left')
matrix['r4'] = round(matrix['r4']/matrix['r1'],2) 

In [14]:
# 角色3天内最大等级 max_role_level
groups = login_info_3d.groupby(['user_id','role_id'])
temp = groups['role_level'].agg([('max_role_level', 'max')]).reset_index()
matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')

In [15]:
# 选取部分字段
login_3d_info = matrix[['user_id', 'role_id','r1', 'r2', 'r3', 'r4', 'max_role_level']]
login_3d_info = login_3d_info.drop_duplicates()
print(login_3d_info.shape)
login_3d_info.to_csv('./FE_data/login_3d_info.csv', index=False)

(6135258, 7)


In [17]:
product_login = pd.merge(login_3d_info, product_info_3d, on=['user_id', 'role_id'], how='left', validate='one_to_one')
product_login.fillna(0, inplace=True)
product_login.to_csv('./FE_data/product_login.csv', index=None)
del product_login, login_3d_info, login_info_3d, matrix, matrix_1, temp, product_info_3d
gc.collect()

NameError: name 'login_3d_info' is not defined